In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import (
    QFTGate,              # Quantum Fourier Transform
    real_amplitudes,   # VQE ansatz
    efficient_su2,     # VQE ansatz (more expressive)
    n_local,         # Custom ansatz builder
    XGate, YGate, ZGate, HGate  # Standard gates
)
import numpy as np

print("✅ Imports successful!")

## Part 1: QFT (Quantum Fourier Transform) - EXAM CRITICAL!

In [ ]:
# Create QFT
qft3 = QFTGate(3)

# Create a circuit to visualize the QFT
qc = QuantumCircuit(3)
qc.append(qft3, range(3))

print("QFT on 3 qubits:")
print(qc.draw())
print(f"\nNumber of qubits: {qft3.num_qubits}")

print("\n✅ QFT is a pre-built circuit!")

In [ ]:
# QFT with inverse
iqft3 = QFTGate(3).inverse()
print("Inverse QFT:")


# QFT followed by inverse = identity
qc = QuantumCircuit(3)
qc.compose(qft3, inplace=True)
qc.compose(iqft3, inplace=True)

print("\nQFT + Inverse QFT (returns to initial state):")
print(qc.draw())

In [ ]:
# QFT in larger circuit
qc = QuantumCircuit(3)
qc.h(range(3))  # Prepare superposition
qc.append(QFTGate(3), range(3))  # Apply QFT

print("Circuit with QFT:")
print(qc.draw())

print("\n🎯 EXAM: QFT used in Shor's algorithm & quantum phase estimation!")

### ⚠️ EXAM TRAP: QFT Properties

```python
# Common exam questions:
qft4 = QFTGate(4)
# Q: How many qubits? A: 4
# Q: How to get inverse? A: QFTGate(4).inverse()
# Q: Used in which algorithms? A: Shor's, QPE, HHL
```

## Part 2: RealAmplitudes - VQE Ansatz (GUARANTEED EXAM!)

In [ ]:
# Create RealAmplitudes ansatz
ansatz = real_amplitudes(num_qubits=3, reps=2)

print("RealAmplitudes(3 qubits, 2 reps):")
print(ansatz.draw())
print(f"\nParameters: {list(ansatz.parameters)[:5]}...")
print(f"Total parameters: {ansatz.num_parameters}")
print(f"Depth: {ansatz.depth()}")
# Explain why depth 7
# RealAmplitudes structure for 3 qubits, 2 reps:
# Layer 0: RY gates on all 3 qubits (parallel, depth +1)
# Entangling layer: 2 CNOT gates in linear pattern (sequential, depth +2)
# Layer 1: RY gates on all 3 qubits (parallel, depth +1)
# Entangling layer: 2 CNOT gates in linear pattern (sequential, depth +2)
# Layer 2: RY gates on all 3 qubits (parallel, depth +1)
# Total depth: 1 + 2 + 1 + 2 + 1 = 7

print("\n📐 Depth breakdown:")
print("- 3 rotation layers (RY on all qubits in parallel): 3 × 1 = 3")
print("- 2 entangling layers (2 CNOTs sequential): 2 × 2 = 4")
print("- Total: 3 + 4 = 7")
print("\n✅ Depth = (reps + 1) + (2 × reps) for linear entanglement!")

### RealAmplitudes Structure:

1. **Rotation Layer** - RY gates on all qubits
2. **Entangling Layer** - CNOT gates
3. **Repeat** for `reps` times
4. **Final Rotation Layer**

In [ ]:
# Effect of reps parameter
print("Effect of reps:")
for reps in [1, 2, 3]:
    ansatz_temp = real_amplitudes(num_qubits=2, reps=reps)
    print(f"reps={reps}: {ansatz_temp.num_parameters} parameters, depth={ansatz_temp.depth()}")

print("\n✅ More reps = more parameters = more expressivity!")

In [ ]:
# Entanglement patterns
print("Linear entanglement (default):")
ansatz_linear = real_amplitudes(3, entanglement='linear', reps=1)
print(ansatz_linear.draw())

print("\nFull entanglement:")
ansatz_full = real_amplitudes(3, entanglement='full', reps=1)
print(ansatz_full.draw())

print("\n🎯 EXAM: Know entanglement patterns!")

In [ ]:
# Binding parameters (VQE pattern)
ansatz = real_amplitudes(2, reps=1)
print("Before binding:")
print(ansatz.draw())

# Bind with random values
param_values = np.random.random(ansatz.num_parameters) * 2 * np.pi
ansatz_bound = ansatz.assign_parameters(param_values)

print("\nAfter binding:")
print(ansatz_bound.draw())
print("\n✅ Ready for VQE optimization!")

## Part 3: EfficientSU2 - More Expressive Ansatz

In [ ]:
# Create EfficientSU2
ansatz = efficient_su2(num_qubits=3, reps=2)

print("EfficientSU2(3 qubits, 2 reps):")
print(ansatz.draw())
print(f"\nTotal parameters: {ansatz.num_parameters}")

In [ ]:
# Compare with RealAmplitudes
n_qubits = 3
reps = 2

real_amp = real_amplitudes(n_qubits, reps=reps)
eff_su2 = efficient_su2(n_qubits, reps=reps)

print("Comparison:")
print(f"RealAmplitudes: {real_amp.num_parameters} parameters")
print(f"EfficientSU2:   {eff_su2.num_parameters} parameters")
print("\n✅ EfficientSU2 has MORE parameters (uses both RY and RZ)!")

### RealAmplitudes vs EfficientSU2

| Feature | RealAmplitudes | EfficientSU2 |
|---------|----------------|---------------|
| Rotation gates | RY only | RY + RZ |
| Parameters | Fewer | More |
| Expressivity | Lower | Higher |
| Use case | Simple VQE | Complex VQE |

**EXAM TIP**: More parameters = more expressivity but harder to optimize!

## Part 4: TwoLocal - Custom Ansatz Builder

In [ ]:
# TwoLocal with custom gates
ansatz = n_local(
    num_qubits=3,
    rotation_blocks='ry',        # Single-qubit gates
    entanglement_blocks='cz',    # Two-qubit gates
    reps=2
)

print("TwoLocal(RY + CZ):")
print(ansatz.draw())

In [ ]:
# Multiple rotation gates
ansatz2 = n_local(
    num_qubits=2,
    rotation_blocks=['ry', 'rz'],  # Both RY and RZ
    entanglement_blocks='cx',
    reps=1
)

print("TwoLocal([RY, RZ] + CX):")
print(ansatz2.draw())
print("\n✅ TwoLocal gives full control over ansatz structure!")

### TwoLocal Pattern:

```python
TwoLocal(
    num_qubits=n,
    rotation_blocks='ry',      # or ['ry', 'rz']
    entanglement_blocks='cx',  # or 'cz', 'swap'
    reps=k
)
```

**EXAM TIP**: TwoLocal is the general template for RealAmplitudes and EfficientSU2!

## Part 5: Standard Gates Library

In [ ]:
# Import gates as objects
from qiskit.circuit.library import XGate, YGate, ZGate, HGate, CXGate

qc = QuantumCircuit(2)

# Append gate objects
qc.append(HGate(), [0])
qc.append(XGate(), [1])
qc.append(CXGate(), [0,1])

print("Circuit with gate objects:")
print(qc.draw())

print("\n✅ Can use gates as objects with .append()!")

## 📝 Practice Questions

### Question 1: QFT Usage

**How do you create a 4-qubit inverse QFT?**

A) `QFT(4).inverse()`  
B) `QFTGate(4).inverse()`  
C) `InverseQFT(4)`  
D) `QFT(4).dagger()`

<details>
<summary>Answer</summary>

**B) `QFTGate(4).inverse()`**

```python
# Correct way
iqft = QFT(4, inverse=True)

# Alternative (also works)
iqft = QFTGate(4).inverse()
```

Both work, but `inverse=True` parameter is more direct.
</details>

---

### Question 2: RealAmplitudes Parameters

**How many parameters in `RealAmplitudes(num_qubits=3, reps=2)`?**

A) 6  
B) 9  
C) 12  
D) 15

<details>
<summary>Answer</summary>

**B) 9**

Formula: `num_qubits * (reps + 1)`

```python
# RealAmplitudes structure:
# Layer 0: 3 RY gates (3 params)
# Entangling: 2 CX gates (no params)
# Layer 1: 3 RY gates (3 params)
# Entangling: 2 CX gates (no params)
# Layer 2: 3 RY gates (3 params)
# Total: 3 + 3 + 3 = 9 parameters
```

reps=2 means 2 entangling layers, which creates 3 rotation layers!
</details>

---

### Question 3: Ansatz Comparison

**Which ansatz has MORE parameters for same num_qubits and reps?**

A) RealAmplitudes  
B) EfficientSU2  
C) They have the same  
D) Depends on entanglement pattern

<details>
<summary>Answer</summary>

**B) EfficientSU2**

```python
# For 3 qubits, 2 reps:
real = RealAmplitudes(3, reps=2)
eff = EfficientSU2(3, reps=2)

print(real.num_parameters)  # 9 (only RY)
print(eff.num_parameters)   # 18 (RY + RZ)
```

EfficientSU2 uses both RY and RZ rotations, doubling parameters!
</details>

---

### Question 4: TwoLocal Pattern

**What does `rotation_blocks` specify in TwoLocal?**

A) Two-qubit entangling gates  
B) Single-qubit rotation gates  
C) Number of repetitions  
D) Entanglement pattern

<details>
<summary>Answer</summary>

**B) Single-qubit rotation gates**

```python
TwoLocal(
    num_qubits=2,
    rotation_blocks='ry',      # Single-qubit gates ✓
    entanglement_blocks='cx',  # Two-qubit gates
    reps=1
)
```

- `rotation_blocks` = single-qubit gates (ry, rz, rx, etc.)
- `entanglement_blocks` = two-qubit gates (cx, cz, swap, etc.)
</details>

---

## ✅ Key Takeaways

### Core Concepts

1. **QFT (Quantum Fourier Transform)**
   - `QFT(n)` creates n-qubit QFT
   - `inverse=True` for inverse QFT
   - Used in: Shor's, QPE, HHL

2. **RealAmplitudes** - VQE ansatz (simple)
   - `RealAmplitudes(num_qubits=n, reps=k)`
   - Only RY rotations
   - Parameters: `n * (k + 1)`
   - Entanglement: 'linear', 'full', 'circular'

3. **EfficientSU2** - VQE ansatz (expressive)
   - Uses RY and RZ rotations
   - More parameters than RealAmplitudes
   - More expressivity

4. **TwoLocal** - Custom ansatz builder
   - `rotation_blocks`: single-qubit gates
   - `entanglement_blocks`: two-qubit gates
   - General template for ansätze

### Critical Exam Facts

- ✅ QFT used in quantum algorithms (Shor's, QPE)
- ✅ RealAmplitudes: RY rotations only
- ✅ EfficientSU2: RY + RZ rotations
- ✅ More reps = more parameters = deeper circuit
- ✅ TwoLocal is general template
- ✅ All ansätze have parameterized circuits
- ✅ Must bind parameters before execution

### Exam Patterns

**Q**: How to create QFT?  
**A**: `QFTGate(n)` for n qubits

**Q**: VQE ansatz with simple rotations?  
**A**: `real_amplitudes(n_qubits, reps=k)`

**Q**: More expressive ansatz?  
**A**: `efficient_su2(n_qubits, reps=k)`

**Q**: Custom ansatz structure?  
**A**: `n_local(n, rotation_blocks, entanglement_blocks, reps)`

### Comparison Table

| Ansatz | Rotations | Parameters (n=3, reps=2) | Use Case |
|--------|-----------|--------------------------|----------|
| RealAmplitudes | RY | 9 | Simple VQE |
| EfficientSU2 | RY + RZ | 18 | Expressive VQE |
| TwoLocal | Custom | Varies | Custom ansatz |

### Mnemonic

🧠 **"QFT for Algorithms, RealAmplitudes for VQE!"**

**Next**: Classical Control (c_if, conditional operations)!